# Задание 2

К вам поступила информация о числе заказов за прошедшие **3 месяца** с разрешением по неделям. Постройте (если это возможно) прогноз продаж на следующие **3 месяца**, с учётом того, что в неделю с **2020-02-02** по **2020-02-09** была проведена массивная акция, повысившая число заказов на **7%**.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import date
from fbprophet import Prophet
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected = True)

ModuleNotFoundError: No module named 'fbprophet'

### Открытие csv-файла и проверка его содержимого

In [ ]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/426638/test2_completed.csv')

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(10,8))
sns.lineplot(data = df, x='Дата', y='Число заказов')

Исходный датафрейм является слишком маленьким для точного предсказания, но можно попробовать спрогнозировать приблизительный тренд развития с помощью Facebook Prophet. Для этого необходимо привести исходный датафрейм к определенному виду.

### Подготовка датафрейма для Facebook Prophet

Преобразуем названия колонок в стандартизированные. Из функции выше видно, что колонка **'Дата'** имеет тип данных **object**. Её также необходимо привести к виду **datetime**:

In [ ]:
df = df.rename(columns={'Дата':'ds', 'Число заказов':'y'})
df['ds'] = pd.to_datetime(df.ds)

### Подготовка модели Facebook Prophet

Поскольку у нас имеется всего 14 строк с данными, не вижу смысла тренировать модель на выборке из них. Будем использовать весь датафрейм. У нас нет необходимой разбивки данных по часам, дням или месяцам, поэтому передать сезонность в параметры модели не представляется возможным. А так как количество наблюдений захватывает лишь малый период от года, то включать праздничные и выходные дни тоже нецелесообразно. 
Учтем проведенную промоакцию **с 02-02-2020 по 09-02-2020**, увеличившую количество заказов на **7%**. Учитывая разбивку данных по неделям, возьмем за итоговую дату события последний ее день **09-02-2020**. Для этого заведем отдельный датафрейм, в который передадим стандартизированное значение промо-акции:

In [ ]:
promo_action = pd.DataFrame({'holiday':'promo_action', 'ds':pd.to_datetime(['2020-02-09'])})

In [ ]:
promo_action

In [ ]:
predict_model = Prophet(holidays=promo_action)
predict_model.fit(df)
predictions=13

In [ ]:
future = predict_model.make_future_dataframe(periods=predictions, freq='W')
forecast = predict_model.predict(future)

In [ ]:
predict_model.plot(forecast)

### Построение интерактивного графика

In [ ]:
iplot([
    go.Scatter(x=df['ds'], y=df['y'], name='fact'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='prediction'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='trend')
])

### Расчет средней ошибки модели предсказания

In [ ]:

cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MAPE (средняя абсолютная ошибка в процентах) – ', np.mean(abs(cmp_df[:-predictions]['p'])),'%')
print('MAE (средняя абсолютная ошибка) – ', np.mean(abs(cmp_df[:-predictions]['e'])))

### Необходимые значения предсказаний на ближайшие 3 месяца

In [ ]:
forecast[['ds', 'trend', 'yhat_lower','yhat_upper']][14:]

## Выводы

* Наблюдается линейный восходящий тренд продаж товара;
* Проведенная акция не сильно повлияла на скорость восхождения линейного тренда;
* Зависимость тренда от сезонности остается неизвестной, необходимо собрать данные за год;
* Необходимо более точно изучить с чем связан рост количества заказов после 22 марта. При таком маленьком количестве данных, он сильно влияет на тренд. 
